# Implementando o modelo binomial para a precificação de opções

Aluno: Bruno Marcelino

### Importando dados

In [2]:
import pandas as pd
import numpy as np

In [3]:
dados = pd.read_excel("dados.xlsx")
dados.drop(0, axis = 0, inplace = True)

### Criando variáveis

In [4]:
dados["up"] = np.where(dados["R BBDC4"] >= 0, dados["R BBDC4"], 0)
dados["q"] = np.where(dados["R BBDC4"] >= 0, 1, 0)
dados["down"] = np.where(dados["R BBDC4"] < 0, dados["R BBDC4"], 0)
dados["1-q"] = np.where(dados["R BBDC4"] >= 0, 0, 1)

q = sum(dados["q"])/len(dados["q"])
q_c = 1-q
u = dados[dados["up"] != 0].mean()["up"]
d = dados[dados["down"] != 0].mean()["down"]

dados.head()

,Date,BBDC4,R BBDC4,up,q,down,1-q
1,2021-05-01 00:00:00,23.863636,-0.0068,0.0000,0,-0.0068,1
2,2021-06-01 00:00:00,24.654545,0.0326,0.0326,1,0.0000,0
3,2021-07-01 00:00:00,25.309090,0.0262,0.0262,1,0.0000,0
4,2021-08-01 00:00:00,25.090908,-0.0087,0.0000,0,-0.0087,1
5,2021-11-01 00:00:00,24.645454,-0.0179,0.0000,0,-0.0179,1


### Entradas

In [5]:
r = 0.0915 # risk-free
S = 22.6 # preço atual
X = 23.17 # strike da opção
dt = 15 # dias úteis até o vencimento

### Probabilidades de cada cenário

In [6]:
mults, probs = [], []  
for i in range(dt+1):
    exp_u = i
    exp_d = dt-i
    
    mult = ((1+u)**exp_u)*((1+d)**exp_d)
    mults.append(mult)
    
    prob = ((q)**exp_u)*((q_c)**exp_d)
    probs.append(prob)

In [7]:
def tri_pascal(n):
    linhas = [[1], [1,1]]
    for i in range(n-1):
        linha_ant = linhas[-1]

        linha_nova = [1]
        for j in range(i+1):
            num = linha_ant[j] + linha_ant[j+1]
            linha_nova.append(num)
        linha_nova.append(1)   

        linhas.append(linha_nova)
    return linhas

probabilidades = []
for i in range(len(probs)):
    probabilidades.append(probs[i] * tri_pascal(15)[-1][i])

### Esperança para o preço da call

In [8]:
precos = [i*S for i in mults]
resultado = [X-i for i in precos]
precos_calls = [i if i > 0 else 0 for i in resultado]

### Valor do prêmio da call hoje

In [9]:
C = 0
for i in range(len(precos_calls)):
    C += precos_calls[i]*probabilidades[i] 
C = C * np.exp(-r*dt/21)

In [11]:
print("Ticker da Opção".ljust(30), "BBDCB234")
print("Preço Spot do Ativo".ljust(30), "R$ {:2.2f}".format(S))
print("Strike da Opção".ljust(30), "R$ {:2.2f}".format(X))
print("Dias-úteis até o vencimento".ljust(30), str(dt).center(9))
print("Taxa Livre de Risco".ljust(30), "{:.2%}\n".format(r).center(9))

print("Preço Alvo da Opção".ljust(30), "R$ {:5.2f}".format(C))
print("Preço Atual da Opção".ljust(30), "R$ {:5.2f}".format(0.44))
print("Resultado Alvo da Operação".ljust(30), "{:.2%}".format((C/0.44)-1).center(9))

Ticker da Opção                BBDCB234
Preço Spot do Ativo            R$ 22.60
Strike da Opção                R$ 23.17
Dias-úteis até o vencimento        15   
Taxa Livre de Risco              9.15%
 
Preço Alvo da Opção            R$  0.82
Preço Atual da Opção           R$  0.44
Resultado Alvo da Operação       87.35% 
